In [ ]:
import pymongo as pm 
import pprint
import datetime
from datetime import timedelta, timezone
import matplotlib.dates as mdates
import time
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
db_add = "<Your DB Address>"
auth_source = "<Auth Source>"
username = "<Your Username>"
password = "<Your Password>"

client=pm.MongoClient(db_add,
                      ssl=True,
                      username=username,
                      password=password,
                      authSource = auth_source,
                      authMechanism='SCRAM-SHA-1',
                      tlsAllowInvalidCertificates = True
                     )

db = client['carsharing'] #choosing DB

In [ ]:
ActiveBookings = db['ActiveBookings']
ActiveParkings = db['ActiveParkings']

PermanentBookings = db['PermanentBookings']
PermanentParkings = db['PermanentParkings']

enjoy_ActiveBookings = db['enjoy_ActiveBookings']
enjoy_ActiveParkings = db['enjoy_ActiveParkings']

enjoy_PermanentBookings = db['enjoy_PermanentBookings']
enjoy_PermanentParkings = db['enjoy_PermanentParkings']

In [ ]:
########################## Task 1.1 ##########################

print("In ActiveBookings there are", ActiveBookings.count_documents({}), "documents")
print("In ActiveParkings there are", ActiveParkings.count_documents({}), "documents")

print("In PermanentBookings there are", PermanentBookings.count_documents({}), "documents")
print("In PermanentParkings there are", PermanentParkings.count_documents({}), "documents")

print("In enjoy_ActiveBookings there are", enjoy_ActiveBookings.count_documents({}), "documents")
print("In enjoy_ActiveParkings there are", enjoy_ActiveParkings.count_documents({}), "documents")

print("In enjoy_PermanentBookings there are", enjoy_PermanentBookings.count_documents({}), "documents")
print("In enjoy_PermanentParkings there are", enjoy_PermanentParkings.count_documents({}), "documents")

In [ ]:
########################## Task 1.2 ##########################

print("Car2go collects data from: ", PermanentBookings.distinct("city"))
print("Enjoy collects data from: ", enjoy_PermanentBookings.distinct("city"))

In [ ]:
########################## Task 1.4 & 1.5 ##########################

first_car2go = PermanentBookings.find({}, {"city": 1, "init_time": 1, "init_date": 1, "_id": 0}).sort("init_time", 1).limit(1)
last_car2go = PermanentBookings.find({}, {"city": 1, "init_time": 1, "init_date": 1, "_id": 0}).sort("init_time", -1).limit(1)
list_car2go = list(first_car2go) + list(last_car2go)

print("\nCar2Go:")
print("\nFirst measurement:", list_car2go[0]["city"], "\nTimestamp:", list_car2go[0]["init_time"],
      "\nGMT:", datetime.fromtimestamp(list_car2go[0]["init_time"]).strftime('%Y-%m-%d %H:%M:%S'),
      "at GMT+1. \nLocal timezone:", list_car2go[0]["init_date"])
print("\nLast measurement:", list_car2go[1]["city"], "\nTimestamp:", list_car2go[1]["init_time"],
      "\nGMT:", datetime.fromtimestamp(list_car2go[1]["init_time"]).strftime('%Y-%m-%d %H:%M:%S'),
      "at GMT+1. \nLocal timezone:", list_car2go[1]["init_date"])


first_enjoy = enjoy_PermanentBookings.find({}, {"city": 1, "init_time": 1, "init_date": 1, "_id": 0}).sort("init_time", 1).limit(1)
last_enjoy = enjoy_PermanentBookings.find({}, {"city": 1, "init_time": 1, "init_date": 1, "_id": 0}).sort("init_time", -1).limit(1)
list_enjoy = list(first_enjoy) + list(last_enjoy)

print("\nEnjoy:")
print("\nFirst measurement:", list_enjoy[0]["city"], "\nTimestamp:", list_enjoy[0]["init_time"],
      "\nGMT:", datetime.fromtimestamp(list_enjoy[0]["init_time"]).strftime('%Y-%m-%d %H:%M:%S'),
      "at GMT+1. \nLocal timezone:", list_enjoy[0]["init_date"])

print("\nLast measurement:", list_enjoy[1]["city"], "\nTimestamp:", list_enjoy[1]["init_time"],
      "\nGMT:", datetime.fromtimestamp(list_enjoy[1]["init_time"]).strftime('%Y-%m-%d %H:%M:%S'),
      "at GMT+1. \nLocal timezone:", list_enjoy[1]["init_date"])

In [ ]:
########################## Task 1.6 ##########################
######### Berlin #########
Berlin_cars_PermanentBookings = PermanentBookings.distinct("plate", {"city": "Berlin"})
Berlin_cars_ActiveBookings = ActiveBookings.distinct("plate", {"city": "Berlin"})
Berlin_cars_ActiveParkings = ActiveParkings.distinct("plate", {"city": "Berlin"})

print("In Berlin,", len(Berlin_cars_PermanentBookings),"car2go vehicles have been listed. \n Only", 
      len(Berlin_cars_ActiveBookings) + len(Berlin_cars_ActiveParkings), "were running as of the last measurement.")

######### Firenze #########
Firenze_cars_PermanentBookings = PermanentBookings.distinct("plate", {"city": "Firenze"})
Firenze_cars_ActiveBookings = ActiveBookings.distinct("plate", {"city": "Firenze"})
Firenze_cars_ActiveParkings = ActiveParkings.distinct("plate", {"city": "Firenze"})

print("In Firenze,", len(Firenze_cars_PermanentBookings),"car2go vehicles have been listed. \n Only", 
      len(Firenze_cars_ActiveBookings) + len(Firenze_cars_ActiveParkings), "were running as of the last measurement.")

######### Toronto #########
Toronto_cars_PermanentBookings = PermanentBookings.distinct("plate", {"city": "Toronto"})
Toronto_cars_ActiveBookings = ActiveBookings.distinct("plate", {"city": "Toronto"})
Toronto_cars_ActiveParkings = ActiveParkings.distinct("plate", {"city": "Toronto"})

print("In Toronto,", len(Toronto_cars_PermanentBookings),"car2go vehicles have been listed. \n Only", 
      len(Toronto_cars_ActiveBookings) + len(Toronto_cars_ActiveParkings), "were running as of the last measurement.")

In [ ]:
########################## Task 1.7 ##########################

jan2018_start=int('1514764800')
jan2018_end=int('1517439600')

pipeline = [
            {"$match": {
                "init_time": {"$gte": jan2018_start, "$lte": jan2018_end},
                "$or": [{"city": "Berlin"},{"city": "Firenze"}, {"city": "Toronto"}]
                }
              },
            {"$project": {         
                "_id": 0,
                "city": 1
                }
              },
            {"$group": {         
                "_id": "$city",
                "tot_rentals": {"$sum":1}
                }

              }
        ]
result = PermanentBookings.aggregate(pipeline)

print("The number of rentals per city in January 2018 are:\n")
for i in result:
    print("{0}: {1}".format(i["_id"], i["tot_rentals"]))

In [ ]:
########################## Task 1.8 ##########################

modes = ["walking.duration", "public_transport.duration", "driving.duration"]

######### Berlin #########
alternative_Berlin = PermanentBookings.count_documents(
    {"$and": [{"city": "Berlin"},
              { "$and": [
                  {"walking.duration":{"$ne":-1}},
                  {"public_transport.duration":{"$ne":-1}},
                  {"driving.duration":{"$ne":-1}}
              ]
              }
             ]
    })             
print("Berlin Bookings alternative transportation all modes recorder: ",alternative_Berlin)

one_alternative_Berlin = PermanentBookings.count_documents(
    {"$and": [{"city": "Berlin"},
              { "$or": [
                  {"walking.duration":{"$ne":-1}},
                  {"public_transport.duration":{"$ne":-1}},
                  {"driving.duration":{"$ne":-1}}
              ]
              }
             ]
    })
print("Berlin Bookings alternative transportation at least one of modes recorder: ", one_alternative_Berlin)
for mode in modes:
    alt_mod = PermanentBookings.count_documents({"city": "Berlin", mode: {"$ne": -1}})
    print(mode.split(".")[0] + " mode: " + str(alt_mod))
    
######### Firenze #########
alternative_Firenze = PermanentBookings.count_documents(
    {"$and": [{"city": "Firenze"},
              { "$and": [
                  {"walking.duration":{"$ne":-1}},
                  {"public_transport.duration":{"$ne":-1}},
                  {"driving.duration":{"$ne":-1}}
              ]
              }
             ]
    })             
print("Firenze Bookings alternative transportation all modes recorder: ",alternative_Firenze)

one_alternative_Firenze = PermanentBookings.count_documents(
    {"$and": [{"city": "Firenze"},
              { "$or": [
                  {"walking.duration":{"$ne":-1}},
                  {"public_transport.duration":{"$ne":-1}},
                  {"driving.duration":{"$ne":-1}}
              ]
              }
             ]
    })
print("Firenze Bookings alternative transportation at least one of modes recorder: ", one_alternative_Firenze)
for mode in modes:
    alt_mod = PermanentBookings.count_documents({"city": "Firenze", mode: {"$ne": -1}})
    print(mode.split(".")[0] + " mode: " + str(alt_mod))
    
######### Toronto #########
alternative_Toronto = PermanentBookings.count_documents(
    {"$and": [{"city": "Toronto"},
              { "$and": [
                  {"walking.duration":{"$ne":-1}},
                  {"public_transport.duration":{"$ne":-1}},
                  {"driving.duration":{"$ne":-1}}
              ]
              }
             ]
    })             
print("Toronto Bookings alternative transportation all modes recorder: ",alternative_Toronto)

one_alternative_Toronto = PermanentBookings.count_documents(
    {"$and": [{"city": "Berlin"},
              { "$or": [
                  {"walking.duration":{"$ne":-1}},
                  {"public_transport.duration":{"$ne":-1}},
                  {"driving.duration":{"$ne":-1}}
              ]
              }
             ]
    })
print("Toronto Bookings alternative transportation at least one of modes recorder: ", one_alternative_Toronto)
for mode in modes:
    alt_mod = PermanentBookings.count_documents({"city": "Berlin", mode: {"$ne": -1}})
    print(mode.split(".")[0] + " mode: " + str(alt_mod))

In [ ]:
########################## Task 2.1 a&b ##########################

Dec2017_start=int('1512082800')
jan2018_end=int('1517439600')

def duration(city):
    pipeline = [
        {"$match": {
            "city": city,
            "init_time": {"$gte": Dec2017_start, "$lte": jan2018_end}
            }
        },
        {"$project": {
            "_id": 0,
            "city": 1,
            "duration": {
                "$ceil": {
                    "$divide": [{"$subtract": ["$final_time", "$init_time"]}, 60]
                    }
                }
            }
        },
        {"$group": {
            "_id": "$duration"
            }
        }
    ]
    return pipeline

def plot_Booking_Parking(city_dict):
    fig, axs = plt.subplots(1, 1, figsize=(5, 5))
    axs.hist(city_dict['bookings'],
             bins=1000,
             density=True,
             histtype='step',
             cumulative=True,
             label='Bookings',
             color='blue')
    axs.set_xscale('log')
    
    axs.hist(city_dict['parkings'],
             bins=1000,
            density=True,
            histtype='step',
            cumulative=True,
            label='Parkings',
            color='red')
    axs.set_xscale('log')
    
    axs.legend(loc='upper left')
    axs.set_title('CDF of booking/parkings in '+ str(city_dict['city']), fontsize=16)
    axs.set_xlabel('Duration [min]', fontsize=14)
    axs.set_ylabel('CDF', fontsize=14)
    axs.grid(True, which='both')
    plt.tight_layout()
    plt.savefig('CDF_Booking_Parking_'+str(city_dict['city'])+'.png', dpi=1200)
    plt.show()
    
######### CDF #########
    
cities = ["Berlin","Firenze","Toronto"]
for city in cities:
    city_dict = {'city': city, 'bookings':[], 'parkings':[]}
    pipeline = duration(city)
    coll_booking=PermanentBookings
    coll_parking=PermanentParkings
    
    # bookings
    bookings_duration=coll_booking.aggregate(pipeline)
    city_dict['bookings'] =[d['_id'] for d in list(bookings_duration)]
    # parkings
    parking_duration=coll_parking.aggregate(pipeline)
    city_dict['parkings']=[d['_id'] for d in list(parking_duration)]
    plot_Booking_Parking(city_dict)

In [ ]:
########################## Task 2.1 c ##########################

Dec2017_start=int('1512082800')
jan2018_end=int('1517439600')

def WeekDayDuration(city, day):
    pipeline = [
        {"$match": {
            "city": city,
            "init_time": {"$gte": Dec2017_start, "$lte": jan2018_end}
            }
        },
        {"$project": {
            "_id": 0,
            "city": 1,
            "duration": {
                "$ceil": {
                    "$divide": [{"$subtract": ["$final_time", "$init_time"]}, 60]
                    }
                },
            "WeekDay": {
                "$dayOfWeek": "$init_date"
                },
            }
        },
        {"$match": {
            "WeekDay": day
            }
        },
        {"$group": {
            "_id": "$duration"
            }
        },
        {"$sort": {
            "_id": 1
            }
        }
    ]
    return pipeline

######### Toronto booking #########
city="Toronto"
plt.figure(figsize=(6,4))

for day in range(1, 8):
    bookings_duration=PermanentBookings.aggregate(WeekDayDuration(city, day))
    bookings=[d["_id"] for d in list(bookings_duration)]

    plt.hist(bookings,
             bins=1000,
             density=True,
             histtype='step',
             cumulative=True)
    plt.xscale('log')
    
plt.legend(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], loc='upper left')
plt.title('CDF of Booking duration per day of the week in '+city, fontsize=13)
plt.xlabel('Duration [min]', fontsize=12)
plt.ylabel('CDF', fontsize=12)
plt.grid(True, which='both')
plt.tight_layout()
plt.savefig('CDF_week_'+city+'_book.png', dpi=1200)
plt.show()

######### Toronto parking #########

plt.figure(figsize=(6,4))

for day in range(1, 8):
    parkings_duration=PermanentParkings.aggregate(WeekDayDuration(city, day))
    parkings=[d["_id"] for d in list(parkings_duration)]

    plt.hist(parkings,
             bins=1000,
             density=True,
             histtype='step',
             cumulative=True)
    plt.xscale('log')
    
plt.legend(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], loc='upper left')
plt.title('CDF of Parking duration per day of the week in '+city, fontsize=13)
plt.xlabel('Duration [min]', fontsize=12)
plt.ylabel('CDF', fontsize=12)
plt.grid(True, which='both')
plt.tight_layout()
plt.savefig('CDF_week_'+city+'_park.png', dpi=1200)
plt.show()

In [ ]:
########################## Task 2.2 - Booking ##########################

cities = ["Berlin", "Firenze", "Toronto"]

for city in cities:
    Dec2017_start = int('1512082800')
    jan2018_end = int('1517439600')
    matrix = np.zeros((1440, 2))
    plt.figure(figsize=(12, 6))

    PermanentBookings = db['PermanentBookings']
    for i in range(1440):
        matrix[i, 0] = i
        matrix[i, 1] = PermanentBookings.count_documents(
            {'$and': [
                {"city": city},
                {"init_time": {
                    '$gte': Dec2017_start, '$lt': (Dec2017_start + 3600)}
                }
            ]}
        )
        Dec2017_start = Dec2017_start + 3600

    plt.plot(matrix[:, 1], color='red')
    plt.xlabel('Days')
    plt.ylabel('Number of bookings')
    plt.grid()
    plt.title('Bookings per hour of the day - ' + city)
    plt.grid(True, which="both")

    x_ticks_labels = []
    for j in range(31):
        x_ticks_labels.append(str(j + 1) + '/12/17')
    for j in range(31):
        x_ticks_labels.append(str(j + 1) + '/1/18')

    x_ticks = np.arange(0, 1441, step=23)[:len(x_ticks_labels)]
    plt.xticks(x_ticks, x_ticks_labels, rotation=270)
    plt.tick_params('x', labelrotation=270, length=6, width=1, grid_alpha=0.5, labelsize=8)
    plt.savefig('overtime_' + city + '.png', dpi=1200)
    plt.show()


In [ ]:
########################## Task 2.2 - Parking ##########################

cities = ["Berlin", "Firenze", "Toronto"]

for city in cities:
    Dec2017_start = int('1512082800')
    jan2018_end = int('1517439600')
    matrix = np.zeros((1440, 2))
    plt.figure(figsize=(12, 6))

    PermanentParkings = db['PermanentParkings']
    for i in range(1440):
        matrix[i, 0] = i
        matrix[i, 1] = PermanentParkings.count_documents(
            {'$and': [
                {"city": city},
                {"init_time": {
                    '$gte': Dec2017_start, '$lt': (Dec2017_start + 3600)}
                }
            ]}
        )
        Dec2017_start = Dec2017_start + 3600

    plt.plot(matrix[:, 1], color='blue')
    plt.xlabel('Days')
    plt.ylabel('Number of Parkings')
    plt.grid()
    plt.title('Parkings per hour of the day - ' + city)
    plt.grid(True, which="both")

    x_ticks_labels = []
    for j in range(31):
        x_ticks_labels.append(str(j + 1) + '/12/17')
    for j in range(31):
        x_ticks_labels.append(str(j + 1) + '/1/18')

    x_ticks = np.arange(0, 1441, step=23)[:len(x_ticks_labels)]
    plt.xticks(x_ticks, x_ticks_labels, rotation=270)
    plt.tick_params('x', labelrotation=270, length=6, width=1, grid_alpha=0.5, labelsize=8)
    plt.savefig('overtimep_' + city + '.png', dpi=1200)
    plt.show()


In [ ]:
########################## Task 2.3 - Booking ##########################
city = 'Berlin'
PermanentBookings = db['PermanentBookings']
Dec2017_start = 1512086400
jan2018_end = 1517443199

pipeline = [
    {"$match": {
        "city": city,
        "init_time": {"$gte": Dec2017_start, "$lte": jan2018_end},
        "final_time": {"$gte": Dec2017_start, "$lte": jan2018_end}
    }},
    {"$project": {
        "hour": {"$hour": "$init_date"},
        "day": {"$dayOfMonth": "$init_date"},
        "month": {"$month": "$init_date"},
        "duration": {"$divide": [{"$subtract": ["$final_time", "$init_time"]}, 60]}
    }},
    {"$group": {
        "_id": {"day": "$day", "hour": "$hour", "month": "$month"},
        "totalBookings": {"$sum": 1},
        "average_duration": {"$avg": "$duration"},
        "st_Dev_duration": {"$stdDevPop": "$duration"}
    }}
]

Bookings_per_dayHours_no_filtered = list(PermanentBookings.aggregate(pipeline))

Bookings_per_dayHours_no_filtered.sort(key=lambda x: (x["_id"]['month'], x["_id"]['day']))

daysIndex1 = [0]
currentDay1 = [f"{Bookings_per_dayHours_no_filtered[0]['_id']['day']}/{Bookings_per_dayHours_no_filtered[0]['_id']['month']}"]

for it, el in enumerate(Bookings_per_dayHours_no_filtered):
    if it != 0 and (el["_id"]["day"], el["_id"]["month"]) != (currentDay1[-1].split('/')[0], currentDay1[-1].split('/')[1]):
        currentDay1.append(f"{el['_id']['day']}/{el['_id']['month']}")
        daysIndex1.append(it)

indexes = np.arange(len(Bookings_per_dayHours_no_filtered))
totalBookings = [el["totalBookings"] for el in Bookings_per_dayHours_no_filtered]

pipeline_f = [
    {"$match": {
        "city": city,
        "init_time": {"$gte": Dec2017_start, "$lte": jan2018_end},
        "final_time": {"$gte": Dec2017_start, "$lte": jan2018_end}
    }},
    {"$project": {
        "hour": {"$hour": "$init_date"},
        "day": {"$dayOfMonth": "$init_date"},
        "month": {"$month": "$init_date"},
        "duration": {"$divide": [{"$subtract": ["$final_time", "$init_time"]}, 60]},
        "origin_coord_lat": {"$arrayElemAt": [{"$arrayElemAt": ["$origin_destination.coordinates", 0]}, 0]},
        "origin_coord_lon": {"$arrayElemAt": [{"$arrayElemAt": ["$origin_destination.coordinates", 0]}, 1]},
        "destination_coord_lat": {"$arrayElemAt": [{"$arrayElemAt": ["$origin_destination.coordinates", 1]}, 0]},
        "destination_coord_lon": {"$arrayElemAt": [{"$arrayElemAt": ["$origin_destination.coordinates", 1]}, 1]}
    }},
    {"$group": {
        "_id": {"day": "$day", "month": "$month", "hour": "$hour"},
        "totalBookings": {"$sum": 1},
    }}
]
Bookings_per_dayHours_filtered = list(PermanentBookings.aggregate(pipeline_f))
Bookings_per_dayHours_filtered.sort(key=lambda x: (x["_id"]['month'], x["_id"]['day']))
totalBookings_filtered = [el["totalBookings"] for el in Bookings_per_dayHours_filtered]

plt.figure(figsize=(12, 6))
plt.plot(indexes, totalBookings, label="No filtered bookings",linewidth=0.7)
plt.plot(indexes, totalBookings_filtered, label="Filtered bookings",linewidth=0.7)
plt.grid()
x_ticks_labels = []
for j in range(31):
    x_ticks_labels.append(str(j + 1) + '/12/17')
for j in range(31):
    x_ticks_labels.append(str(j + 1) + '/1/18')

x_ticks = np.arange(0, 1441, step=23)[:len(x_ticks_labels)]
plt.xticks(x_ticks, x_ticks_labels, rotation=270)
plt.tick_params('x', labelrotation=270, length=6, width=1, grid_alpha=0.5, labelsize=8)
plt.legend()
plt.xlabel('Day')
plt.ylabel('Rentals')
plt.title('Number of Bookings per hour and day in Berlin')
plt.savefig('filterb_' + city + '.png', dpi=1200)
plt.show()

In [ ]:
########################## Task 2.3 - Parking ##########################
city = 'Berlin'
PermanentParkings = db['PermanentParkings']
Dec2017_start = 1512086400
jan2018_end = 1517443199

pipeline = [
    {"$match": {
        "city": city,
        "init_time": {"$gte": Dec2017_start, "$lte": jan2018_end},
        "final_time": {"$gte": Dec2017_start, "$lte": jan2018_end}
    }},
    {"$project": {
        "hour": {"$hour": "$init_date"},
        "day": {"$dayOfMonth": "$init_date"},
        "month": {"$month": "$init_date"},
        "duration": {"$divide": [{"$subtract": ["$final_time", "$init_time"]}, 60]}
    }},
    {"$group": {
        "_id": {"day": "$day", "hour": "$hour", "month": "$month"},
        "totalParkings": {"$sum": 1},
        "average_duration": {"$avg": "$duration"},
        "st_Dev_duration": {"$stdDevPop": "$duration"}
    }}
]

Parkings_per_dayHours_no_filtered = list(PermanentParkings.aggregate(pipeline))

Parkings_per_dayHours_no_filtered.sort(key=lambda x: (x["_id"]['month'], x["_id"]['day']))

daysIndex1 = [0]
currentDay1 = [f"{Parkings_per_dayHours_no_filtered[0]['_id']['day']}/{Parkings_per_dayHours_no_filtered[0]['_id']['month']}"]

for it, el in enumerate(Parkings_per_dayHours_no_filtered):
    if it != 0 and (el["_id"]["day"], el["_id"]["month"]) != (currentDay1[-1].split('/')[0], currentDay1[-1].split('/')[1]):
        currentDay1.append(f"{el['_id']['day']}/{el['_id']['month']}")
        daysIndex1.append(it)

indexes = np.arange(len(Parkings_per_dayHours_no_filtered))
totalParkings = [el["totalParkings"] for el in Parkings_per_dayHours_no_filtered]

pipeline_f = [
    {"$match": {
        "city": city,
        "init_time": {"$gte": Dec2017_start, "$lte": jan2018_end},
        "final_time": {"$gte": Dec2017_start, "$lte": jan2018_end}
    }},
    {"$project": {
        "hour": {"$hour": "$init_date"},
        "day": {"$dayOfMonth": "$init_date"},
        "month": {"$month": "$init_date"},
        "duration": {"$divide": [{"$subtract": ["$final_time", "$init_time"]}, 60]},
        "origin_coord_lat": {"$arrayElemAt": [{"$arrayElemAt": ["$origin_destination.coordinates", 0]}, 0]},
        "origin_coord_lon": {"$arrayElemAt": [{"$arrayElemAt": ["$origin_destination.coordinates", 0]}, 1]},
        "destination_coord_lat": {"$arrayElemAt": [{"$arrayElemAt": ["$origin_destination.coordinates", 1]}, 0]},
        "destination_coord_lon": {"$arrayElemAt": [{"$arrayElemAt": ["$origin_destination.coordinates", 1]}, 1]}
    }},
    {"$group": {
        "_id": {"day": "$day", "month": "$month", "hour": "$hour"},
        "totalParkings": {"$sum": 1},
    }}
]
Parkings_per_dayHours_filtered = list(PermanentParkings.aggregate(pipeline_f))
Parkings_per_dayHours_filtered.sort(key=lambda x: (x["_id"]['month'], x["_id"]['day']))
totalParkings_filtered = [el["totalParkings"] for el in Parkings_per_dayHours_filtered]

plt.figure(figsize=(12, 6))
plt.plot(indexes, totalParkings, label="No filtered Parkings",linewidth=0.7)
plt.plot(indexes, totalParkings_filtered, label="Filtered Parkings",linewidth=0.7)
plt.grid()
x_ticks_labels = []
for j in range(31):
    x_ticks_labels.append(str(j + 1) + '/12/17')
for j in range(31):
    x_ticks_labels.append(str(j + 1) + '/1/18')

x_ticks = np.arange(0, 1441, step=23)[:len(x_ticks_labels)]
plt.xticks(x_ticks, x_ticks_labels, rotation=270)
plt.tick_params('x', labelrotation=270, length=6, width=1, grid_alpha=0.5, labelsize=8)
plt.legend()
plt.xlabel('Day')
plt.ylabel('Rentals')
plt.title('Number of Parkings per hour and day in Berlin')
plt.savefig('filter_' + city + '.png', dpi=1200)
plt.show()

In [ ]:
########################## Task 2.4 - Booking ##########################

cities = ['Berlin', 'Firenze', 'Toronto']

PermanentBookings = db['PermanentBookings']

Dec2017_start = 1512086400
jan2018_end = 1517443199

plt.figure(figsize=(12, 6))

for city in cities:
    pipeline_f = [
        {"$match": {
            "city": city,
            "init_time": {"$gte": Dec2017_start, "$lte": jan2018_end},
            "final_time": {"$gte": Dec2017_start, "$lte": jan2018_end}
        }},
        {"$project": {
            "hour": {"$hour": "$init_date"},
            "day": {"$dayOfMonth": "$init_date"},
            "month": {"$month": "$init_date"},
            "duration": {"$divide": [{"$subtract": ["$final_time", "$init_time"]}, 60]},
            "origin_coord_lat": {"$arrayElemAt": [{"$arrayElemAt": ["$origin_destination.coordinates", 0]}, 0]},
            "origin_coord_lon": {"$arrayElemAt": [{"$arrayElemAt": ["$origin_destination.coordinates", 0]}, 1]},
            "destination_coord_lat": {"$arrayElemAt": [{"$arrayElemAt": ["$origin_destination.coordinates", 1]}, 0]},
            "destination_coord_lon": {"$arrayElemAt": [{"$arrayElemAt": ["$origin_destination.coordinates", 1]}, 1]}
        }},
        {"$group": {
            "_id": {"day": "$day", "month": "$month", "hour": "$hour"},
            "totalBookings": {"$sum": 1},
        }}
    ]
    
    # Fetch  and Sort the data from the database
    Bookings_per_dayHours_filtered = list(PermanentBookings.aggregate(pipeline_f))
    Bookings_per_dayHours_filtered.sort(key=lambda x: (x["_id"]['month'], x["_id"]['day']))
    
    totalBookings_filtered = [el["totalBookings"] for el in Bookings_per_dayHours_filtered]

    plt.plot(totalBookings_filtered, label=f"Filtered bookings ({city})",linewidth=0.7)

x_ticks_labels = []
for j in range(31):
    x_ticks_labels.append(str(j + 1) + '/12/17')
for j in range(31):
    x_ticks_labels.append(str(j + 1) + '/1/18')

x_ticks = np.arange(0, 1441, step=23)[:len(x_ticks_labels)]
plt.xticks(x_ticks, x_ticks_labels, rotation=270)
plt.tick_params('x', labelrotation=270, length=6, width=1, grid_alpha=0.5, labelsize=8)

plt.grid()
plt.legend()
plt.xlabel('Day')
plt.ylabel('Rentals')
plt.title('Number of Filtered Bookings per Hour and Day')
plt.savefig('filtered_bookings.png', dpi=1200)
plt.show()

In [ ]:
########################## Task 2.4 - Parking ##########################

cities = ['Berlin', 'Firenze', 'Toronto']

PermanentParkings = db['PermanentParkings']

Dec2017_start = 1512086400
jan2018_end = 1517443199

plt.figure(figsize=(12, 6))

for city in cities:
    pipeline_f = [
        {"$match": {
            "city": city,
            "init_time": {"$gte": Dec2017_start, "$lte": jan2018_end},
            "final_time": {"$gte": Dec2017_start, "$lte": jan2018_end}
        }},
        {"$project": {
            "hour": {"$hour": "$init_date"},
            "day": {"$dayOfMonth": "$init_date"},
            "month": {"$month": "$init_date"},
            "duration": {"$divide": [{"$subtract": ["$final_time", "$init_time"]}, 60]},
            "origin_coord_lat": {"$arrayElemAt": [{"$arrayElemAt": ["$origin_destination.coordinates", 0]}, 0]},
            "origin_coord_lon": {"$arrayElemAt": [{"$arrayElemAt": ["$origin_destination.coordinates", 0]}, 1]},
            "destination_coord_lat": {"$arrayElemAt": [{"$arrayElemAt": ["$origin_destination.coordinates", 1]}, 0]},
            "destination_coord_lon": {"$arrayElemAt": [{"$arrayElemAt": ["$origin_destination.coordinates", 1]}, 1]}
        }},
        {"$group": {
            "_id": {"day": "$day", "month": "$month", "hour": "$hour"},
            "totalParkings": {"$sum": 1},
        }}
    ]
    
    # Fetch and Sort the data from the database
    Parkings_per_dayHours_filtered = list(PermanentParkings.aggregate(pipeline_f))
    Parkings_per_dayHours_filtered.sort(key=lambda x: (x["_id"]['month'], x["_id"]['day']))
    
    totalParkings_filtered = [el["totalParkings"] for el in Parkings_per_dayHours_filtered]

    plt.plot(totalParkings_filtered, label=f"Filtered parkings ({city})",linewidth=0.7)

x_ticks_labels = []
for j in range(31):
    x_ticks_labels.append(str(j + 1) + '/12/17')
for j in range(31):
    x_ticks_labels.append(str(j + 1) + '/1/18')

x_ticks = np.arange(0, 1441, step=23)[:len(x_ticks_labels)]
plt.xticks(x_ticks, x_ticks_labels, rotation=270)
plt.tick_params('x', labelrotation=270, length=6, width=1, grid_alpha=0.5, labelsize=8)

plt.grid()
plt.legend()
plt.xlabel('Day')
plt.ylabel('Parked cars')
plt.title('Number of Filtered Parkings per Hour and Day')
plt.savefig('filtered_Parkings.png', dpi=1200)
plt.show()

In [ ]:
########################## Task 2.5 - Booking ##########################
city = "Toronto"
Dec2017_start = int('1512086400')
jan2018_end=int('1517439600')
PermanentBookings = db['PermanentBookings']

Bookings_per_hours_date_filtered = list(PermanentBookings.aggregate([
                    {
                    "$match":{
                        "city": f"{city}",
                        "init_time":{"$gte":Dec2017_start,"$lte":jan2018_end},
                        "final_time":{"$gte":Dec2017_start,"$lte":jan2018_end},
                        }
                    },
                    {
                    "$project":{
                        "hour": {"$hour": "$init_date"},
                        "day": {"$dayOfMonth": "$init_date"},
                        "month": {"$month": "$init_date"},                        
                        "duration":{"$divide" : [{"$subtract":["$final_time","$init_time"]},60]},
                        "origin_coord_lat":{"$arrayElemAt": [ {"$arrayElemAt":["$origin_destination.coordinates",0]}, 0]},
                        "origin_coord_lon":{"$arrayElemAt": [ {"$arrayElemAt":["$origin_destination.coordinates",0]}, 1]},
                        "destination_coord_lat":{"$arrayElemAt": [ {"$arrayElemAt":["$origin_destination.coordinates",1]},0]},
                        "destination_coord_lon":{"$arrayElemAt": [ {"$arrayElemAt":["$origin_destination.coordinates",1]},1]}
                        }
                    },
                    {
                    "$project":{
                        "hour":1,
                        "day":1,
                        "month": 1,
                        "duration":1,
                        "moved_lat": {"$abs": {"$subtract":["$origin_coord_lat","$destination_coord_lat"]}},
                        "moved_lon": {"$abs": {"$subtract":["$origin_coord_lon","$destination_coord_lon"]}},
                        }
                    },
                    {
                    "$match":{
                        "moved_lat":{"$gte":0.0005},
                        "moved_lon":{"$gte":0.0005},
                        "duration":{"$lte":180, "$gte":5}
                        }
                    },
                    {
                    "$sort":{"duration":1}
                    },
                    {
                    "$group":{"_id": {"day":"$day", "month": "$month"},
                              "list_values": {"$push": "$duration"}}
                    },
                    {
                    "$project":{
                        "day":1,
                        "month": 1,
                        "average":{"$avg":"$list_values"},
                        "percentile_90":{"$arrayElemAt":["$list_values",
                                                         {"$floor":{ "$multiply": [0.9,{"$size": "$list_values"}]}}]},
                                                          "median":{"$arrayElemAt":["$list_values",
                                                         {"$floor":{ "$multiply": [0.5,{"$size": "$list_values"}]}}]},
                                                          "st_Dev":{"$stdDevSamp":"$list_values"}
                            }
                        }
                    ], allowDiskUse=True))

Bookings_per_hours_date_non_filtered = list(PermanentBookings.aggregate([
                    {
                    "$match":{
                        "city": f"{city}",
                        "init_time":{"$gte":Dec2017_start,"$lte":jan2018_end},
                        "final_time":{"$gte":Dec2017_start,"$lte":jan2018_end},
                        }
                    },
                    {
                    "$project":{
                        "hour": { "$hour": "$init_date" },
                        "day": { "$dayOfMonth": "$init_date" },
                        "month": {"$month": "$init_date"},
                        "duration":{"$divide" : [{"$subtract":["$final_time","$init_time"]},60]}
                        }
                    },
                    {
                    "$sort":{"duration":1}
                    },
                    {
                    "$group":{"_id": {"day":"$day", "month": "$month"},"list_values": { "$push": "$duration" }}
                    },
                    {
                    "$project":{
                        "day":1,
                        "month": 1,
                        "average":{"$avg":"$list_values"},
                        "percentile_90":{"$arrayElemAt":["$list_values",
                        {"$floor":{ "$multiply": [0.9,{"$size": "$list_values"}]}}]},
                        "median":{"$arrayElemAt":["$list_values",
                        {"$floor":{ "$multiply": [0.5,{"$size": "$list_values"}]}}]},
                        "st_Dev":{"$stdDevSamp":"$list_values"},
                        }
                    }
                    ], allowDiskUse=True))

def custom_sort(item):
    month, day = item["_id"]["month"], item["_id"]["day"]
    return month, day

Bookings_per_hours_date_filtered.sort(key=custom_sort)
Bookings_per_hours_date_non_filtered.sort(key=custom_sort)

year = "2017"
prev_day = 0
days = []
for el in Bookings_per_hours_date_filtered:
    current_day = el["_id"]["day"]
    if current_day < prev_day:
        year = "2018"
    days.append(f"{current_day}/{el['_id']['month']}/{year}")
    prev_day = current_day

averages_filtered=[el["average"] for el in Bookings_per_hours_date_filtered]
percentile_90_filtered=[el["percentile_90"] for el in Bookings_per_hours_date_filtered]
median_filtered=[el["median"] for el in Bookings_per_hours_date_filtered]
st_Dev_duration_filtered=[el["st_Dev"] for el in Bookings_per_hours_date_filtered]

averages_non_filtered=[el["average"] for el in Bookings_per_hours_date_non_filtered]
percentile_90_non_filtered=[el["percentile_90"] for el in Bookings_per_hours_date_non_filtered]
median_non_filtered=[el["median"] for el in Bookings_per_hours_date_non_filtered]
st_Dev_duration_non_filtered=[el["st_Dev"] for el in Bookings_per_hours_date_non_filtered]

plt.figure(figsize=(8,6))
plt.plot(days,averages_filtered,'r-.',label="Avg_fil")
plt.plot(days,percentile_90_filtered,'b-.',label="Per_90_fil")
plt.plot(days,median_filtered,'g-.',label="Med_fil")
plt.plot(days,st_Dev_duration_filtered,'k-.',label="Std_dev_fil")

plt.grid()
plt.legend()
plt.xticks(rotation=270, ha="right")
plt.tick_params('x', labelrotation=270, length=10, width=1, grid_alpha=0.5, labelsize=8)
plt.xlabel('Days')
plt.ylabel('Duration')
plt.title('Statistics of duration bookings, {}'.format(city))
plt.savefig('stat_' + city + '.png', dpi=1200)
plt.show()

In [ ]:
########################## Task 2.6 ##########################

def parked_november(collection, ct, h_start=6, h_end=12, d_start=1, d_end=5, min_duration=5, m=11, y=2017):
# Query to extract the coordinates of each car.
    number_doc_nov = db[collection].aggregate([
        {'$match': {'city': ct}},
        {'$project': {
        '_id': 0,
        'duration': {'$divide': [{'$subtract': ['$final_time', '$init_time']}, 60]},
        'plate': 1,
        'city': 1,
        'init_time': 1,
        'init_date': 1,
        'year': {'$year': '$init_date'},
        'month': {'$month': '$init_date'},
        'day': {'$dayOfWeek': '$init_date'},
        'hour': {'$hour': '$init_date'},
        'longitude': {
        '$arrayElemAt': ["$loc.coordinates", 0]},
        'latitude': {
        '$arrayElemAt': ["$loc.coordinates", 1]}
        }},
        {'$match': {'month': m, 'year': y, 'duration': {'$gte': min_duration}}},
        {'$match': {'month': m, 'year': y, 'day': {'$gte': d_start, '$lte': d_end},
        'hour': {'$gte': h_start, '$lt': h_end}, 'duration': {'$gte': min_duration}}}
])

    df = pd.DataFrame(number_doc_nov) # dataframe was created with the extracted data.
    r_earth = 6378 # size of the earth in kilometers
    dx = dy = 0.5 # delta of distance in kilometers
    lat = df['latitude'].min() # minimum latitude of all the data
    long = df['longitude'].min() # minimum longitude of all the data
    new_lat = lat # initial value
    new_long = long # initial value
    n_car_zone = np.zeros(len(df)) # initial value vector that contains the number of cars
    # in the zone
    id_zone = np.zeros(len(df)) # number of each zone
    num_zone = 0
    # Divide into zones
    while new_lat <= df['latitude'].max():
        new_lat = lat + (dy / r_earth) * (180 / np.pi)
        while new_long <= df['longitude'].max():
            new_long = long + (dx / r_earth) * (180 / np.pi) / np.cos(lat * np.pi / 180)
            n_car_zone[df[(long <= df['longitude']) & (df['longitude'] < new_long)
                & (lat <= df['latitude']) & (
                    df['latitude'] < new_lat)].index] = \
                        df[(long <= df['longitude']) & (df['longitude'] < new_long)
                        & (lat <= df['latitude'])& (df['latitude'] < new_lat)]['hour'].count() / 0.25
            id_zone[df[(long <= df['longitude']) & (df['longitude'] < new_long)
                & (lat <= df['latitude'])
                & (df['latitude'] < new_lat)].index] = num_zone
            if (df[
                (long <= df['longitude']) & (df['longitude'] < new_long) &
                (lat <= df['latitude'])
                & (df['latitude'] < new_lat)]['hour'].count() / (dx * dx)) != 0:
                num_zone = num_zone + 1
            long = new_long
        lat = new_lat
        long = df['longitude'].min()
        new_long = long
    df['N_cars'] = n_car_zone # add density of car as a new column
    df['Zone'] = id_zone # add an identification for each zone
    df.to_csv(ct + "_d_" + str(d_start) + "_to_" + str(d_end) + "_h_" +
        str(h_start) + "_to_" + str(h_end) + "_" + str(dx) + "_km.csv")

In [ ]:
def booking_november(collection, ct, m=11, y=2017, min_duration=5, max_duration=(3 * 60)):
    number_doc_nov = db[collection].aggregate([
        {'$match': {'city': ct}},
        {'$project': {
        '_id': 0,
        'moved': {'$ne': [
        {'$arrayElemAt': ["$origin_destination.coordinates", 0]},
        {'$arrayElemAt': ["$origin_destination.coordinates", 1]}
        ]},
        'duration': {'$divide': [{'$subtract': ['$final_time', '$init_time']}, 60]},
        'city': 1,
        'init_time': 1,
        'init_date': 1,
        'year': {'$year': '$init_date'},
        'month': {'$month': '$init_date'},
        'day': {'$dayOfWeek': '$init_date'},
        'hour': {'$hour': '$init_date'},
        'longitude_org': {
        '$arrayElemAt': [{'$arrayElemAt': ["$origin_destination.coordinates", 0]}, 0]},
        'latitude_org': {
        '$arrayElemAt': [{'$arrayElemAt': ["$origin_destination.coordinates", 0]}, 1]},
        'longitude_des': {
        '$arrayElemAt': [{'$arrayElemAt': ["$origin_destination.coordinates", 1]}, 0]},
        'latitude_des': {
        '$arrayElemAt': [{'$arrayElemAt': ["$origin_destination.coordinates", 1]}, 1]},
        }},
        {'$match': {'month': m, 'year': y,
        'moved': True, 'duration': {'$gte': min_duration, '$lte': max_duration}}}
    ])
    df = pd.DataFrame(number_doc_nov)
    r_earth = 6378 # size of the earth in kilometers
    dx = dy = 0.5 # delta of distance in kilometers
    if df['latitude_org'].min() < df['latitude_des'].min(): # Minimum latitude of all the data
        min_lat = df['latitude_org'].min()
    else:
        min_lat = df['latitude_des'].min()
    if df['longitude_org'].min() < df['longitude_des'].min(): # Minimum longitude of all the data
        min_long = df['longitude_org'].min()
    else:
        min_long = df['longitude_des'].min()
    new_lat = min_lat # Initial value
    new_long = min_long # Initial value
    lat = min_lat
    long = min_long
    id_zone = np.zeros([len(df), 2]) # Number of each zone
    num_zone = 0
    if df['latitude_org'].max() > df['latitude_des'].max(): # Maximum latitude of all the data
        max_lat = df['latitude_org'].max()
    else:
        max_lat = df['latitude_des'].max()
    if df['longitude_org'].max() > df['longitude_des'].max(): # Maximum longitude of all the data
        max_long = df['longitude_org'].max()
    else:
        max_long = df['longitude_des'].max()
    # Divide into zones
    while new_lat <= max_lat:
        new_lat = lat + (dy / r_earth) * (180 / np.pi)
        while new_long <= max_long:
            new_long = long + (dx / r_earth) * (180 / np.pi) / np.cos(lat * np.pi / 180)
            id_zone[
                df[(long <= df['longitude_org']) & (df['longitude_org'] < new_long)
                    & (lat <= df['latitude_org']) &
                    (df['latitude_org'] < new_lat)].index, 0] = num_zone
            id_zone[df[(long <= df['longitude_des']) & (df['longitude_des'] < new_long)
                & (lat <= df['latitude_des']) & (
                df['latitude_des'] < new_lat)].index, 1] = num_zone
            if ((df[
                    (long <= df['longitude_org']) & (df['longitude_org'] < new_long)
                     & (lat <= df['latitude_org']) & (
                    df['latitude_org'] < new_lat)]['hour'].count() / 0.25) != 0) | \
                    ((df[(long <= df['longitude_des']) & (df['longitude_des'] < new_long)
                    & (lat <= df['latitude_des']) &
                    (df['latitude_des'] < new_lat)]['hour'].count() / 0.25) != 0):
                num_zone = num_zone + 1
            long = new_long
        lat = new_lat
        long = min_long
        new_long = long
    df['Zone_org'] = id_zone[:, 0] # Add identification for each zone
    df['Zone_des'] = id_zone[:, 1] # Add identification for each zone
    return df

In [ ]:
# Generation of files
parked_november('PermanentParkings', 'Toronto')
parked_november('PermanentParkings', 'Toronto', 12, 18)
parked_november('PermanentParkings', 'Toronto', 18, 24)
parked_november('PermanentParkings', 'Toronto', 6, 12, 6, 7)
parked_november('PermanentParkings', 'Toronto', 12, 18, 6, 7)
parked_november('PermanentParkings', 'Toronto', 18, 24, 6, 7)
parked_november('PermanentParkings', 'Toronto', 6, 12, 1, 7)
parked_november('PermanentParkings', 'Toronto', 12, 18, 1, 7)
parked_november('PermanentParkings', 'Toronto', 18, 24, 1, 7)
# Generation of dataframe for origin-destination matrix
df = booking_november('PermanentBookings', 'Toronto')
# Plot origin-destination matrix
org_des = np.zeros([int(df['Zone_org'].max()) + 1, int(df['Zone_org'].max()) + 1])
for i in range(int(df['Zone_org'].max()) + 1):
    for j in range(int(df['Zone_org'].max()) + 1):
        org_des[i, j] = df[(df['Zone_org'] == i) & (df['Zone_des'] == j)]['hour'].count()
plt.imshow(org_des)
plt.colorbar()
plt.title('Matrix origin-destination', fontsize=15)
plt.ylabel('Origin zone', fontsize=15)
plt.xlabel('Destination zone', fontsize=15)
plt.show()

In [ ]:
import folium
import pandas as pd
from selenium import webdriver
import random

file = 'Firenze_d_1_to_5_h_18_to_24_0.5_km'
path = "path"

data = pd.read_csv(path + file + ".csv")

locations = data[['latitude', 'longitude']]

m = folium.Map(location=[locations['latitude'].mean(), locations['longitude'].mean()], zoom_start=12)

for _, location in locations.iterrows():
    folium.CircleMarker(
        location=[location['latitude'], location['longitude']],
        radius=3,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.25
    ).add_to(m)

m.save('map3.html')
m

In [ ]:
file = 'Firenze_d_1_to_5_h_6_to_12_0.5_km'
path = "path"

data1 = pd.read_csv(path + file + ".csv")

colors = []
for i in range(141):
    colors.append('#%06X' % randint(0, 0xFFFFFF))

m = folium.Map(location=[data1['latitude'].mean(), data1['longitude'].mean()], zoom_start=12)

zone = 1

for col in colors:
    locations1 = data1[['latitude', 'longitude']][data1['Zone'] == zone]
    for _, location in locations1.iterrows():
        folium.CircleMarker(
            location=[location['latitude'], location['longitude']],
            radius=3,
            color=col,
            fill=True,
            fill_color=col,
            fill_opacity=0.5
        ).add_to(m)
    zone += 1

m.save('map4.html')